In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
%%capture
!pip install nyaggle

In [ ]:
df_train = pd.read_feather('../input/optiver-df2/X.f')

In [ ]:
y = pd.read_feather('../input/optiver-df2/y.f')

In [ ]:
n_th = int(0.5*len(df_train))
x_train = df_train.iloc[:n_th].copy()
x_valid = df_train.iloc[n_th:].copy()

x_train.head()

In [ ]:
sns.distplot(y[:n_th].sample(50000).dropna(), label='train')
sns.distplot(y[n_th:].sample(50000).dropna(), label='valid')
plt.legend()
plt.show()

## Show features by adversarial validation

In [ ]:
from nyaggle.validation import adversarial_validate
from lightgbm import LGBMClassifier

drop_features = [
    'trade_150.tau_cluster20_sv_mean_cluster40_size_mean',
    'trade_150.tau_cluster20_sv_mean_cluster40_p2_mean',
    'trade_150.tau_cluster20_sv_mean_cluster20_size_mean',
    'trade_150.tau_cluster20_sv_mean_cluster20_p2_mean',
    'trade_150.tau_cluster20_sv_mean_cluster40_p1_mean',
    'trade_150.tau_cluster20_sv_mean_cluster20_p1_mean',

    'book.log_return1.realized_volatility_cluster40_s_amin',
    'book.log_return1.realized_volatility_cluster40_sv_amin',
    'book.log_return1.realized_volatility_cluster40_p2_amin',
    'book.log_return1.realized_volatility_cluster20_sv_amin',
    'book.log_return1.realized_volatility_cluster20_s_amin',
    'book.log_return1.realized_volatility_cluster20_p2_amin',
    
    'book.log_return1.realized_volatility_cluster40_s_std',
    'book.log_return1.realized_volatility_cluster40_sv_std',
    'book.log_return1.realized_volatility_cluster20_s_std',
    'book.log_return1.realized_volatility_cluster20_sv_std',
    'trade.size.sum_cluster40_s_mean',
    'trade.size.sum_cluster20_s_mean',

]

estimator = LGBMClassifier(n_estimators=50, objective='binary', importance_type='gain',
                           random_state=0)

drop_cols = ['stock_id', 'stock_id_emb0', 'stock_id_emb1', 'stock_id_emb2'] + drop_features
ret = adversarial_validate(x_train.drop(drop_cols, axis=1), x_valid.drop(drop_cols, axis=1), estimator=estimator)

In [ ]:
def plot(col):
    import seaborn as sns
    sns.distplot(x_train[col].sample(50000).dropna(), label='train')
    sns.distplot(x_valid[col].sample(50000).dropna(), label='valid')
    plt.legend()
    plt.show()

In [ ]:
for f in ret.importance.feature[:30]:
    plot(f)
    plt.show()

In [ ]:
y_train

In [ ]:
plot('trade.size.sum_cluster20_s_mean')

## Skewed feature

In [ ]:
np.abs(x_train.sample(30000).skew()).sort_values(ascending=False).head(30)

In [ ]:
sns.distplot(x_train['book_450.price_spread.std'])